In [1]:
import torch.nn as nn
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
# input_data
import sys
import os
sys.path.append(os.path.abspath('../src/'))
from data_loader import getDataSet

import glob


In [2]:
xlsx_name_list = glob.glob("../../data_new/*/*.xlsx")
print (len(xlsx_name_list))

146


In [14]:
from tqdm import tqdm

for xlsx_name in xlsx_name_list:
    input_data = pd.read_excel(xlsx_name, sheet_name='Sheet1')
    for column in ['Event_type_眼动形式', 'Average_pupil_diameter_平均瞳孔直径', '0_direction_眼跳方向', 'Average_velocity_平均眼跳时间', 'Peak_velocity_眼跳峰值速度', '0_amplitude_眼跳幅度']:
        if column not in input_data.columns:
            print (xlsx_name)

../../data_new/mqn4-30-30min Metrics(1)/32.xlsx
../../data_new/mqn4-30-30min Metrics(1)/32.xlsx
../../data_new/mqn4-30-30min Metrics(1)/47.xlsx
../../data_new/mqn4-30-30min Metrics(1)/47.xlsx
../../data_new/mqn4-30-30min Metrics(1)/57.xlsx
../../data_new/mqn4-30-30min Metrics(1)/57.xlsx
../../data_new/mqn4-30-30min Metrics(1)/27.xlsx
../../data_new/mqn4-30-30min Metrics(1)/27.xlsx
../../data_new/mqn4-30-30min Metrics(1)/49.xlsx
../../data_new/mqn4-30-30min Metrics(1)/49.xlsx
../../data_new/mqn4-30-30min Metrics(1)/38.xlsx
../../data_new/mqn4-30-30min Metrics(1)/38.xlsx
../../data_new/mqn4-30-30min Metrics(1)/8.xlsx
../../data_new/mqn4-30-30min Metrics(1)/8.xlsx
../../data_new/mqn4-30-30min Metrics(1)/43.xlsx
../../data_new/mqn4-30-30min Metrics(1)/43.xlsx
../../data_new/mqn4-30-30min Metrics(1)/6.xlsx
../../data_new/mqn4-30-30min Metrics(1)/6.xlsx
../../data_new/mqn4-30-30min Metrics(1)/51.xlsx
../../data_new/mqn4-30-30min Metrics(1)/51.xlsx
../../data_new/mqn4-30-30min Metrics(1)/26.x

Regression Task

given Sequence of data 1500 x 6 and outputs are 2 regression value

(you can output a signle feature by change label_feature to ['执行时间'])

In [12]:
PeriodDataSet = getDataSet(task='Regression')
dataset = PeriodDataSet(xlsx_name_list = xlsx_name_list, label_feature=['执行时间', '反应时间'])
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=2)
for xlsx_name, X, Y in dataloader:
    print (xlsx_name, X, Y)

tensor([[[  0.0000,   4.1945, 191.1700, 227.6300, 256.6800,   6.9400],
         [  0.0000,   4.1945, 191.1700, 206.7400, 250.7900,   6.9400],
         [  0.0000,   4.1945, 191.1700, 227.6300, 256.6800,   6.9400],
         ...,
         [  1.0000,   4.1921, 191.0700, 227.4600, 256.4500,   6.9400],
         [  1.0000,   4.1921, 191.0700, 227.4600, 256.4500,   6.9400],
         [  1.0000,   4.1921, 191.0700, 227.4600, 256.4500,   6.9400]]]) tensor([[13140.,  9391.]])
tensor([[[  0.0000,   4.2471, 193.3900, 231.4200, 261.5500,   7.0500],
         [  0.0000,   4.2471, 193.3900, 356.0000, 356.0000,   7.0500],
         [  0.0000,   4.2471, 193.3900, 374.3800, 374.3800,   7.0500],
         ...,
         [  1.0000,   4.2449, 193.2900, 231.2600, 261.3400,   7.0500],
         [  1.0000,   4.2449, 193.2900, 231.2600, 261.3400,   7.0500],
         [  1.0000,   4.2449, 193.2900, 231.2600, 261.3400,   7.0500]]]) tensor([[27256.,  2971.]])


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/weijiesun/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 309, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/home/weijiesun/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/weijiesun/anaconda3/envs/pytorch/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/weijiesun/data/patch_tst/EyeTimePredictor-RT/src/data_loader.py", line 42, in __getitem__
    X = input_data[self.features].values[:self.sequence_length, :].astype(np.float32)  # 取出特征
  File "/home/weijiesun/anaconda3/envs/pytorch/lib/python3.10/site-packages/pandas/core/frame.py", li

Segmentation

given Sequence of data 1500 x 6 and outputs are 1500 binary bits

1 means within '反应时间' to '执行时间' + '反应时间', 0 means outliers

In [4]:
PeriodDataSet = getDataSet(task='Segmentation')
dataset = PeriodDataSet(xlsx_name_list = xlsx_name_list[:5])
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
for X, Y in dataloader:
    print (X, Y)
    break

tensor([[[  0.0000,   4.0769, 331.5100, 168.3100, 197.3000,   7.5500],
         [  0.0000,   4.0769, 202.1600, 195.0200, 218.9600,   7.0200],
         [  1.0000,   4.0678, 202.1600, 195.0200, 218.9600,   7.0200],
         ...,
         [  1.0000,   4.0740, 201.5100, 460.1200, 461.0500,   7.0200],
         [  1.0000,   4.0740, 201.5100, 463.4500, 464.0700,   7.0200],
         [  1.0000,   4.0740, 201.5100, 466.7700, 467.0800,   7.0200]],

        [[  0.0000,   4.2993,  60.4600, 193.5400, 381.9400,   9.7500],
         [  0.0000,   4.2993, 204.4000, 182.0200, 201.6400,   6.1500],
         [  0.0000,   4.2993, 204.4000, 182.0200, 201.6400,   6.1500],
         ...,
         [  1.0000,   4.2993,  46.3400, 173.3900, 175.6500,   7.1800],
         [  1.0000,   4.2993,  39.4900, 172.9900, 174.5000,   7.2200],
         [  1.0000,   4.2993,  32.6400, 172.5900, 173.3400,   7.2700]]]) tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [5]:
train_list, test_list = train_test_split(xlsx_name_list, test_size=0.33, random_state=42)

In [6]:
train_list

['../../data_new/mhs4-30-30min Metrics/19.xlsx',
 '../../data_new/mhs4-30-30min Metrics/52.xlsx',
 '../../data_new/mhs4-30-30min Metrics/24.xlsx',
 '../../data_new/mqn4-30-30min Metrics(1)/33.xlsx',
 '../../data_new/mqn4-30-30min Metrics(1)/48.xlsx',
 '../../data_new/mhs4-30-30min Metrics/9.xlsx',
 '../../data_new/mhs4-30-30min Metrics/21.xlsx',
 '../../data_new/mqn4-30-30min Metrics(1)/11.xlsx',
 '../../data_new/mhs4-30-30min Metrics/46.xlsx',
 '../../data_new/mqn4-30-30min Metrics(1)/38.xlsx',
 '../../data_new/mqn4-30-30min Metrics(1)/37.xlsx',
 '../../data_new/mhs4-30-30min Metrics/28.xlsx',
 '../../data_new/mqn4-30-30min Metrics(1)/4.xlsx',
 '../../data_new/mqn4-30-30min Metrics(1)/15.xlsx',
 '../../data_new/wyz4-30-30min Metrics/51.xlsx',
 '../../data_new/mhs4-30-30min Metrics/36.xlsx',
 '../../data_new/wyz4-30-30min Metrics/60.xlsx',
 '../../data_new/wyz4-30-30min Metrics/3.xlsx',
 '../../data_new/mhs4-30-30min Metrics/44.xlsx',
 '../../data_new/mqn4-30-30min Metrics(1)/23.xlsx',